### Step 1: Load and Inspect the Data





In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('marketing_data.csv')

# Look at the first few rows
print(df.head())






In [ ]:
# Check column names nd data types
print(df.info())

In [ ]:
# Check for missing values
print(df.isnull().sum())

### Step 2: Handle Missing Income Values & Clean Categorical Data

In [ ]:
# Look at unique values in education and marital status
print(df['Education'].unique())
print(df['Marital_Status'].unique())

In [ ]:
# Clear/standardize categories 
df['Education'] = df['Education'].replace({
    '2n Cycle' : 'Second Cycle',
    'Basic' : 'Basic',
    'Graduation' : 'Graduate',
    'PhD' : 'PhD',
    'Master' : 'Master'
})

# Drop odd marital values
df = df[~df['Marital_Status'].isin(['YOLO','Absurd','Alone'])]

In [ ]:
# Clean column names: remove leading/trailing spaces
df.columns = df.columns.str.strip()
print(df.columns.tolist())


In [ ]:
# Fill in missing incomes with the average income for each(Education,Marital_Status) group: 
df['Income'] = df.groupby(['Education', 'Marital_Status'])['Income'].transform(
    lambda x: x.fillna(x.mean()))

### Step 3: Create New Columns
- Prepare the data for meaningful analysis.

In [ ]:
# Step 3.1 : Calculate Customer Age

from datetime import datetime

# Claculate age from birth year
current_year = datetime.now().year
df['Age'] = current_year - df['Year_Birth']

In [ ]:
# Step 3.2: Total Number of Children

# Add Kidhome and Teenhome
df['Total_Children'] = df['Kidhome'] + df['Teenhome']

In [ ]:
# Step 3.3 : Total Spending
## Sum all spending columns: wine,fruit,meat,fish,sweets and gold.

product_columns = ['MntWines', 'MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']
df['Total_Spend'] = df[product_columns].sum(axis=1)

In [ ]:
# Step 3.4 : Total Purchases from all Channels
purchase_columns = ['NumWebPurchases','NumCatalogPurchases','NumStorePurchases']
df['Total_Purchases'] = df[purchase_columns].sum(axis =1)

In [ ]:
print(df[['Age','Total_Children','Total_Spend','Total_Purchases']].describe())

### Step 4 : Visualizations & Outlier Detection.
Using boxplots and histograms to:
- Understand the distribution of values
- Spot any outliers(extreme values)

In [ ]:
# Step 4.1 : Boxplot for Income

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
sns.boxplot(x=df['Income'])
plt.title("Boxplot of Income")
plt.show()

In [ ]:
# Step 4.2: Histogram for Age

plt.figure(figsize = (8,4))
df['Age'].hist(bins =20,edgecolor='black')
plt.title("Distribution of Age")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Step 4.3 : Treat Outliers
## Outliers in Income, Can remove or cap them. For example, to remove the top 5% of Income values:

df = df[df['Income']< df['Income'].quantile(0.95)]

### Step 5: Encoding Categorical Variables
Convert categorical columns like Education and Marital_Status into numbers
Two techniques:
- Ordinal Encoding: when categories have a natural order(e.g. education levels).
- One-Hot Encoding: when categories have no order(e.g. marital status, country).

In [ ]:
# Step 5.1: Ordinal Encode - Education

# Define the order of education levels
education_order = {
    'Basic' : 0,
    'Second Cycle' : 1,
    'Graduate' : 2,
    'Master' : 3,
    'PhD' : 4
}

# Apply the mapping
df['Education_encode'] = df['Education'].map(education_order)

In [ ]:
# Step 5.2: One-Hot Encode - "Marital_Status" and "Country"
# The (drop_first =True) option avoids multicollinearity by removing one dummy column per feature.

# One-hot encode marital status
df = pd.get_dummies(df,columns=['Marital_Status'], drop_first = True)

# One-hot encode Country
df = pd.get_dummies(df,columns=['Country'],drop_first=True)

In [ ]:
# Step 5.3 Check Your Data

print(df[['Education','Education_encode']].head())
print("-"*20)
print(df.filter(like = 'Marital_Status_').head())

### Step 6: Correlation Heatmap
This step help to visually understand how different numeric variables are related.
Strong correlations can hint at imporant relationships(e.g, higher income -> higher spending).


In [ ]:
# Step 6.1: Generate the Heatmap

import seaborn as sns
import matplotlib.pyplot as plt

# Create a correlation matirx for numerical features
corr_matrix = df.corr(numeric_only=True)

# Plot the heatmap
plt.figure(figsize=(20,20))
sns.heatmap(corr_matrix,annot=True,fmt = ".2f",cmap='coolwarm',linewidths =0.5)
plt.title("Correlation Heatmap")
plt.show()


### Step 7: Hypothesis Testing

#### Hypothesis A
Older people prefer in-store shopping(they might be less tech-savvy).
- Check if age is positively correlated with store purchases, and negatively correlated with web/catalog purchases.

In [ ]:
# Plot Age vs StorePurchases
sns.scatterplot(x='Age', y='NumStorePurchases',data=df)
plt.title("Age vs Store Purchase")
plt.xlabel("Age")
plt.ylabel("Store Purchases")
plt.show()

# Plot Age vs Web Purchases
sns.scatterplot(x='Age', y='NumWebPurchases',data =df)
plt.title("Age vs Web Purchases")
plt.xlabel("Age")
plt.ylabel("Web Purchases")
plt.show()

#Check Correlation Coefficients
corr_store = df[['Age','NumStorePurchases']].corr().iloc[0,1]
corr_web =df[['Age','NumWebPurchases']].corr().iloc[0,1]

print("Correlation between Age and Store Purchases:", round(corr_store,2))
print("Correlation between Age and Web Purchase:", round(corr_web,2))

##### Results:
- Correlation between Age and Store Purchases: 0.16
- Correlation between Age and Web Purchase: 0.15
##### Meaning:
- Both correlations are weak positive(values close to 0).
- This means that as age increases, both store and web purchases slightly increase,but not strongly.
##### Interpretation:
- Web purchases also slightly increase with age, which goes against the hypothesis.
##### Final Conclusion:
- Hypothesis A is not strongly supported.
- Older individuals do not show a clear preference for in-store shopping over online - in fact, both store and online purchases slightly increase with age.


This suggests older customer may be comfortable with both shopping channels.

#### Hypothesis B
Customers with children prefer online shopping(due to time constraints).
- Check if (Total_Children) is positively related to Web Purchases.


In [ ]:
# Visualize Total_Children vs Web Purchases(This will show how web purchases vary depending on the number of children.)
sns.boxplot(x='Total_Children', y ='NumWebPurchases', data =df)
plt.title("Children at Home vs web purchases")
plt.xlabel("Total Children")
plt.ylabel("Web Purchases")
plt.show()

# Calculate Correlation
corr_children_web = df[['Total_Children','NumWebPurchases']].corr().iloc[0,1]
print("Correlation between Total Children and Web Purchases:", round(corr_children_web,2))

##### Results:
- Correlation between Total Children and Web Purchases: -0.13
- The boxplot shows that customers with 0 or 1 child tend to have more web purchases that those with 2 or 3 children.

##### Meanings:
- A correlation of -0.13 is weakly negative.
    - This means as the number of children increases, web purchases tend to slightly decrease.
- The boxplot confirms this - families with more children shop online slightly less, not more.

##### Final Conclusion:
Hypothesis Not supported.
Customers with children do not prefer online shopping based on this data.
In fact, they might shop less online, possibly due to:
- Less discretionary time/spending
- Preference for traditional shopping
- Or constraints not captured by the dataset

#### Hypothesis C
Store sales are being cannibalized by online/catalog channels.
- Check if store purchases has a negative correlation with web or catalog purchases.
- Look for negative values in the correlation matrix.

In [ ]:
# Correlation Check
print(df[['NumStorePurchases','NumWebPurchases','NumCatalogPurchases']].corr())

# Visual Pairplot
sns.pairplot(df[['NumStorePurchases','NumWebPurchases','NumCatalogPurchases']])

##### Interpretation
- All correlations are positive, not negative.
- That means: people who buy from stores are also more likely to buy from web and catalog.
- The scatter plots also show upward trends, confirming this.
##### Final Conclusion
Hypothesis is Not supported

There is no evidence of cannibalization.

Instead, customers who are active in one channel tend to be active in multiple channels- they are likely high- engagement shoppers

#### Hypothesis D
Does the U.S. outperform other countries in total purchases?
- Compare "Total_Purchases" across countries,especially focusing on the US.

In [ ]:
# Available Country Columns
print([col for col in df.columns if 'Country_' in col])




In [ ]:
# US customers
us_avg = df[df['Country_US'] == 1]['Total_Purchases'].mean()

# Non-US customers
non_us_avg = df[df['Country_US'] == 0]['Total_Purchases'].mean()

print("Average Total Purchases (US):", round(us_avg,2))
print("Average Total Purchases (Non-US)", round(non_us_avg,12))

##### Interpretation:
- The difference is about 1.18 purchases on average, which is modest, not dramatic.
- This suggests that US customers do purchase slightly more, but the margin isn't very large.
##### Conclusion:
Hypothesis D is weakly supported.

Yes, the US performs slightly better in total purchases, but not significantly enough to say it strongly outperforms the rest of the world.


### Step 8: Campaign and Product Analysis
This includes visual analysis of product performance, campaign effectiveness, demographics and spending behavior

In [ ]:
# Step 8a: Identify the top-performing products and those with the lowest revenue
product_columns = ['MntWines', 'MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']

# Total revenue per product
product_revenue = df[product_columns].sum().sort_values(ascending=False)
print(product_revenue)

product_revenue.plot(kind='bar', title="Total Revenue by Product",ylabel='Amount Spent',figsize=(8,5))
plt.show()

##### Interpretation:
- Top-Performing Product: Wines - highest revenue
- Second Best: Meat Products
- Lowest Revenue: Fruits or Sweets

This tells the marketing team:
- Which products to focus on for upselling and promotions
- Which low-performing items might need repositioning or removal

In [ ]:
# Step 8b: Correlation between customers's age and acceptance rate of the last campaign
# We want to see if older or younger customers were more likely to accept the last campaign.
# The column to use is likely: AcceptedCmp5 (1 = accepted, 0 = not accepted)

# Step1: Correlation
correlation = df[['Age','AcceptedCmp5']].corr().iloc[0,1]
print("Correlation between Age and Acceptance of Last Campaign:", round(correlation,2))

# Step2: Visualization
sns.boxplot(x='AcceptedCmp5',y='Age', data = df)
plt.title("Age vs Last Campaign Acceptance")
plt.xlabel("Accepted Last Campaign(1=Yes,0=No)")
plt.ylabel("Age")
plt.show()

##### Interpretation
- A Correlation of +0.04 is very week and close to zero.
- This means there's no meaningful relationship between a customer's age and their likelihood to accept the last campaign.
- The boxplot supports this - median ages are nearly the same for both groups(accepted vs not accepted).

##### Final Conclusion:
No significant pattern.
Age does not influence campaign acceptance behavior - marketers should focus on other variables(e.g, spending habits, channel preference).

In [ ]:
# Step 8c: Determine the country with the highest number of customers who accepted the last campaign.
# Count how many customers accepted the last campaign(AcceptedCmp5 ==1) from each country.

# Since one-hot encoding is used, the original Country column is gone. 
# 1> Identify all the Country_* columns.
# 2> For each one, count how many rows have:
#    a> Country_X ==1 and
#    b> AcceptedCmp5 == 1


# List Country Columns (Get all one-hot encoded country columns)
Country_columns = [col for col in df.columns if col.startswith('Country_')]
print(Country_columns)

# Count Accepted Customers by Country
accepted_by_country = {}

for country in Country_columns:
    count = df[(df[country] == 1) & (df['AcceptedCmp5']==1)].shape[0]
    accepted_by_country[country.replace('Country_','')]=count

accepted_series = pd.Series(accepted_by_country).sort_values(ascending=False)
print(accepted_series)

# Plot It
accepted_series.plot(kind='bar', title='Accepted Last Campaign by Country', ylabel ='Number of Acceptances', figsize = (10,5))


##### Interpretation:
- Spain(SP) had the highest number of campaign acceptances by a large margin.
- The next closest country(SA) had less then 1/3 of Spain's acceptances.
- Middle East(ME) had no campaign acceptance.

##### Conclusion:
Spain is the top-performing country in terms of customer response to the last campaign.
Marketers may want to analyze why Spain did so well - possible reasons.
- Better Campaign targeting
- Product preferences
- Higher engagement levels



In [ ]:
# Step 8d: Investigate if there is a discernible pattern in the number of children at home and total expenditure.
# If customers with more children spend more or less in total.
# - X-axis: Total_Children
# - Y-axis: Total_Spend(sum of product purchase)

sns.boxplot(x= 'Total_Children',y= 'Total_Spend',data =df)
plt.title("Total Spend vs Number of Children at Home")
plt.xlabel("Total Children")
plt.ylabel("Total Spend")
plt.show()

# Correlation
correlation = df[['Total_Children','Total_Spend']].corr().iloc[0,1]
print("Correlation between Total_Children and Total_Spend:",round(correlation,2))

##### Results:
- Correlation: -0.47(moderate negative)
- Boxplot shows:
    - Customer with 0 children spend the most.
    - Spending declines steadily as the number of children increases.
    - Customers with 2 or 3 children spend significantly less overall.

##### Interpretation:
- A correlation of -0.47 means a moderate negative relationship:
    - As the number of children increases, total spending decreases.
- This trend is visually confirmed by the boxplot.

##### Conclusion:
Yes, there is a clear pattern.

Customers with more children tend to spend less.

This insight may reflect:
- Budget Constraints in large families.
- Different priorities or product needs


In [ ]:
# Step 8e: Analyze the educational background of customers who lodged compaints in the last 2 years
# Understand what education levels are more common among customers who submitted complaints(Complain ==1).

# Filter Customers who complained
Complainers = df[df['Complain']==1]

# Count how many complainers fall into each education level
education_counts = Complainers['Education'].value_counts()
print(education_counts)

# Visualize it
education_counts.plot(kind = 'bar', title= 'Education level of Customers who Complained',ylabel='Number of Complaints', xlabel='Education Lever',figsize =(8,5))
plt.show()

##### Interpretation:
- A large majority of complaints came from Graduate-level customers.
- Customers with higher education levels(Master/PhD)had fewer complaints.
- This may reflect:
    - Different expectations across education levels
    - Or the distribution of education in the total customer base.

##### Final Conclusion:
Graduate-level customers are the most likely to lodge complaints.
Marketers might want to:
- Investigate what issues are raised by this segment
- Improve satisfaction among mid-level educated customers